# Load Past Key Values into State

import coremltools as ct
import torch

In [21]:
import coremltools as ct
import numpy as np
import torch

In [3]:
mlmodel = ct.models.MLModel("vibe_voice_lm_model_seqlen_32.mlpackage/", compute_units=ct.ComputeUnit.CPU_AND_NE)

In [4]:
cache = torch.load("./demo/voices/streaming_model/en-Carter_man.pt", weights_only=False, map_location="cpu")

In [10]:
print("Num layers", len(cache['lm'].past_key_values))
print("cache shape", cache['lm'].past_key_values[0][0].shape)

Num layers 4
cache shape torch.Size([1, 2, 108, 64])


In [13]:
state = mlmodel.make_state()
state_shape = state.read_state("key_cache").shape
state_shape

(4, 2, 2048, 64)

In [25]:
key_cache = torch.cat(cache['lm'].past_key_values.key_cache)
key_cache = key_cache.float().numpy()
print(key_cache.shape)

(4, 2, 108, 64)


In [26]:
state.write_state("key_cache", np.pad(key_cache, ((0, 0), (0, 0), (0, state_shape[2] - key_cache.shape[2]), (0, 0))))